In [ ]:
import os
from PIL import Image

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, InputLayer, Flatten, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import tensorflow_datasets as tfds

In [ ]:
# Define the folder paths
data_dir = "cell_images"

data = []
labelNames = ["Parasitized", "Uninfected"]

# Set parameters for the dataset
img_height = 224
img_width = 224

# Load the training dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",       # Automatically infer labels from subfolder names
    label_mode="int",        # Use integer labels (0 and 1 for binary classification)
    image_size=(img_height, img_width), # Resize images to a uniform size
    batch_size=32,   # Number of images per batch
    shuffle=True             # Shuffle the dataset for training
)

# Create a normalization layer
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Split dataset into training, validation and test sets 
train_split = 0.8
val_split = 0.1
test_split = 0.1

print("Total images in one batch =",len(dataset))

train_dataset = dataset.take(int(train_split * len(dataset)))
print("Number of training images =", len(train_dataset))

val_test_dataset = dataset.skip(int(train_split * len(dataset)))
val_dataset = val_test_dataset.take(int(val_split * len(dataset)))
print("Number of validation images =", len(val_dataset))

test_dataset = val_test_dataset.skip(int(val_split * len(dataset)))
print("Number of testing images =", len(test_dataset))

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
import matplotlib.pyplot as plt

# Get one batch of images and labels
for images, labels in train_dataset.take(1):  # Take one batch
    image = images[0].numpy()  # Get the first image in the batch
    label = labels[0].numpy()  # Get the corresponding label
    break

# Display the image
plt.imshow(image)
plt.title(f"Label: {labelNames[label]}")  # Display the label as title
plt.axis("off")
plt.show()


In [ ]:
for images, labels in test_dataset.take(1):  # Take one batch
    first_image = images[30].numpy()  # Extract the first image as a NumPy array
    print("Shape of the first image:", first_image.shape)  # Print its shape
    print("Matrix representation of the first image:\n", first_image)  # Print the matrix
    break

In [ ]:
model = tf.keras.Sequential([
    InputLayer(input_shape = (img_height,img_height,3)),
    
    Conv2D(filters=6, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2),

    Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2),

    Flatten(),
    
    Dense(100, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'relu'),
    BatchNormalization(),
    Dense(1, activation = 'sigmoid'),
])

model.summary()

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = 'accuracy',
)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=60, verbose=1)

In [ ]:
plt.style.use('dark_background')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylim([0,5])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
plt.style.use('dark_background')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model.evaluate(test_dataset)